In [1]:
import random
import csv

In [2]:
random.seed(3)  # Set the random seed so the results are reproducible

# Given data
tasks = {
    "ax-cpt": 16.78833333,
    "cuedTS": 10.12166667,
    "flanker": 6.446666667,
    "go-nogo": 9.471666667,
    "nback": 9.271666667,
    "span": 23.68,
    "spatial_cueing": 12.84666667,
    "spatialTS": 10.12166667,
    "stop_signal": 9.046666667,
    "stroop": 6.446666667,
    "visual_search": 11.08,
}

BATTERY_LIMIT = 60  # in minutes
seen_batteries = set()  # Keep track of generated batteries to avoid duplicates

In [3]:
def generate_batteries():
    all_tasks = list(tasks.keys())
    random.shuffle(all_tasks)
    task_order = all_tasks * 5

    # Insert the survey task randomly into the task_order
    survey_position = random.randint(0, len(task_order))
    task_order.insert(survey_position, "survey")

    batteries = []
    current_battery = []
    current_time = 0

    for task in task_order:
        if current_time + (tasks[task] if task in tasks else 40) <= BATTERY_LIMIT:
            current_battery.append(task)
            current_time += tasks[task] if task in tasks else 40
        else:
            batteries.append(current_battery)
            current_battery = [task]
            current_time = tasks[task] if task in tasks else 40

    if current_battery:  # Add the last battery if there are remaining tasks
        batteries.append(current_battery)

    # Convert batteries to a frozenset so they can be stored in a set
    battery_tuple = tuple(frozenset(battery) for battery in batteries)

    # Check if these batteries have been seen before
    if battery_tuple in seen_batteries:
        return []

    seen_batteries.add(battery_tuple)
    return batteries

def calculate_battery_time(battery):
    return sum(tasks[task] if task in tasks else 40 for task in battery)

def all_tasks_within_desired_range(all_sets):
    # Step 1: Initialize a dictionary to store the positions of each task across the sets
    task_positions = {task: [] for task in tasks}

    # Step 2: Iterate through each set and each battery to capture the position of the first instance of each task
    for battery_set in all_sets:
        position = 1  # Initialize position for each new set
        seen_tasks_in_set = set()  # Keep track of tasks we've already seen in this set
        for battery in battery_set:
            for task in battery:
                if task in task_positions and task not in seen_tasks_in_set:
                    task_positions[task].append(position)
                    seen_tasks_in_set.add(task)  # Mark the task as seen in this set
                position += 1

    # Step 3: Calculate the average position for each task
    task_avg_positions = {}
    for task, positions in task_positions.items():
        task_avg_positions[task] = sum(positions) / len(positions)

    # Step 4: Check if the average position is between 5 and 7 for all tasks
    return all(5 <= avg <= 7 for avg in task_avg_positions.values()), task_avg_positions


while True:
    all_sets = []

    # Keep trying until 12 batteries are created for each set
    for _ in range(4):
        batteries = []
        while len(batteries) != 12:
            batteries = generate_batteries()
        all_sets.append(batteries)
    flag, task_avg_positions = all_tasks_within_desired_range(all_sets)
    if flag:
        print(task_avg_positions)
        break  # Exit loop if the sets meet the desired criteria

for set in all_sets:
    for battery in set:
        if 'survey' in battery:
            battery.remove('survey')
            battery.append('survey')

def calculate_mean_indices(array_of_arrays):
    from collections import defaultdict
    import numpy as np
    
    # Track the indices for each unique value
    value_indices = defaultdict(list)
    
    for i, block in enumerate(array_of_arrays):
        for j, sublist in enumerate(block):
            for value in sublist:
                value_indices[value].append(i * len(block) + j)
    
    # Calculate the mean index for each value
    mean_indices = {key: np.mean(indices) for key, indices in value_indices.items()}
    
    return mean_indices

# Flattening the array of arrays into a single list to simplify processing
flattened_arrays = []
for item in all_sets:
    if isinstance(item[0], list):  # Check if the first item is a list, indicating nested structure
        for sublist in item:
            flattened_arrays.extend(sublist)
    else:
        flattened_arrays.extend(item)

# Calculating the average index for each color
indexes = {'ax-cpt': [], 'cuedTS': [], 'flanker': [], 'go-nogo': [], 'nback': [], 'span': [], 'spatial_cueing': [], 'spatialTS': [], 'stop_signal': [], 'stroop': [], 'visual_search': [], 'survey': []}
for i, task in enumerate(flattened_arrays):
    indexes[task].append(i / 4 / 5)

average_indexes = {color: sum(indexes[color])/len(indexes[color]) for color in indexes}
print(average_indexes)

# After generating the batteries, write them to a CSV file
with open('batteries.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    writer.writerow(["Set Number", "Battery Number", "Tasks", "Estimated Time (min)"])
    
    # Write the batteries for each set
    for set_num, battery_set in enumerate(all_sets, start=1):
        for i, battery in enumerate(battery_set, start=1):
            time_estimate = calculate_battery_time(battery)
            writer.writerow([f"Set {set_num}", f"Battery {i}", ', '.join(battery), time_estimate])

{'ax-cpt': 5.0, 'cuedTS': 5.0, 'flanker': 7.0, 'go-nogo': 5.75, 'nback': 5.25, 'span': 6.75, 'spatial_cueing': 6.0, 'spatialTS': 5.25, 'stop_signal': 7.0, 'stroop': 6.75, 'visual_search': 6.25}
{'ax-cpt': 5.5175, 'cuedTS': 5.5175, 'flanker': 5.615, 'go-nogo': 5.5525, 'nback': 5.529999999999999, 'span': 5.6075, 'spatial_cueing': 5.5675, 'spatialTS': 5.527500000000001, 'stop_signal': 5.615, 'stroop': 5.6049999999999995, 'visual_search': 5.5775, 'survey': 6.0375}
